The Bayes Optimal Classifier is a probabilistic model that makes the most probable prediction for a new example.It is described using the Bayes Theorem that provides a principled way for calculating a conditional probability. It is also closely related to the Maximum a Posteriori: a probabilistic framework referred to as MAP that finds the most probable hypothesis for a training dataset.
In practice, the Bayes Optimal Classifier is computationally expensive, if not intractable to calculate, and instead, simplifications such as the Gibbs algorithm and Naive Bayes can be used to approximate the outcome.

Naive Bayes classifiers are a family of simple "probabilistic classifiers" based on applying Bayes' theorem with strong (naive) independence assumptions between the features.
The fundamental Naive Bayes assumption is that each feature makes independent and equal contribution to the outcome.We assume that no pair of features are dependentand each feature is given the same weight(or importance). 
Bayes’ Theorem finds the probability of an event occurring given the probability of another event that has already occurred: P(B|A)=[P(A|B)*P(A)]/P(B)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from matplotlib import pyplot
import math
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [4]:
data_frame = pd.read_csv("D:\Term6\Machine Learning\Homework\Iris.csv")


data_frame.insert(5, "Predict", "None")

data_frame

Sepal_Length  Sepal_Width  Petal_Length  Petal_Width           Class  \
0             5.1          3.5           1.4          0.2     Iris-setosa   
1             4.9          3.0           1.4          0.2     Iris-setosa   
2             4.7          3.2           1.3          0.2     Iris-setosa   
3             4.6          3.1           1.5          0.2     Iris-setosa   
4             5.0          3.6           1.4          0.2     Iris-setosa   
..            ...          ...           ...          ...             ...   
145           6.7          3.0           5.2          2.3  Iris-virginica   
146           6.3          2.5           5.0          1.9  Iris-virginica   
147           6.5          3.0           5.2          2.0  Iris-virginica   
148           6.2          3.4           5.4          2.3  Iris-virginica   
149           5.9          3.0           5.1          1.8  Iris-virginica   

    Predict  
0      None  
1      None  
2      None  
3      None  
4      None  
..      ...  
145    None  
146    None  
147    None  
148    None  
149    None  

[150 rows x 6 columns]

first we load the dataset and add a column to predict the class

In [5]:
train_data = data_frame.sample(frac=0.7)
test_data = data_frame.drop(train_data.index)
class_labels = data_frame.Class.unique()

We calculate the mean and std of each class.(We assume the distribution is guassian)

In [6]:
class_0 = train_data.loc[train_data["Class"] == class_labels[0]]
class_1 = train_data.loc[train_data["Class"] == class_labels[1]]
class_2 = train_data.loc[train_data["Class"] == class_labels[2]]

In [7]:
class_0_mean = class_0.mean()
class_1_mean = class_1.mean()
class_2_mean = class_2.mean()
class_0_std = class_0.std()
class_1_std = class_1.std()
class_2_std = class_2.std()

C:\Users\admin\AppData\Local\Temp/ipykernel_1480/299768135.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  class_0_mean = class_0.mean()
C:\Users\admin\AppData\Local\Temp/ipykernel_1480/299768135.py:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  class_1_mean = class_1.mean()
C:\Users\admin\AppData\Local\Temp/ipykernel_1480/299768135.py:3: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  class_2_mean = class_2.mean()
C:\Users\admin\AppData\Local\Temp/ipykernel_1480/299768135.py:4: Fu

We calculate the probabiltiy that a sample belongs to a distribution.

In [8]:
def calculate_probability(single_data, mean, std):
    exponent = math.exp((-1/2) * (((single_data - mean) / std) ** 2))
    GaussianProbability = (1 / ((math.sqrt(2 * math.pi)) * std)) * exponent
    return GaussianProbability

We assume the samples are independant. so we multiply the probability of all samples of the vector

In [9]:
def calculate_class_probability(vector_data, vector_mean, vector_std):
    prob = float(1)
    for i in range(len(vector_data) - 2):
        prob *= calculate_probability(vector_data[i], vector_mean[i], vector_std[i])
    return prob

Now we calculate the probability of train and test data. the predicted label is the class with the maximum probability

In [13]:
for i in train_data.index:
    P_0 = calculate_class_probability(train_data.loc[i], class_0_mean, class_0_std)
    P_1 = calculate_class_probability(train_data.loc[i], class_1_mean, class_1_std)
    P_2 = calculate_class_probability(train_data.loc[i], class_2_mean, class_2_std)
    maxp=max(P_0,P_1,P_2)
    if  P_0==maxp:
        train_data.at[i, "Predict"] = class_labels[0]
    elif P_1==maxp:
        train_data.at[i, "Predict"] = class_labels[1]
    else:
        train_data.at[i, "Predict"] = class_labels[2]

In [14]:
for i in test_data.index:
    P_0 = calculate_class_probability(test_data.loc[i], class_0_mean, class_0_std)
    P_1 = calculate_class_probability(test_data.loc[i], class_1_mean, class_1_std)
    P_2 = calculate_class_probability(test_data.loc[i], class_2_mean, class_2_std)
    maxp=max(P_0,P_1,P_2)
    if P_0==maxp:
        test_data.at[i, "Predict"] = class_labels[0]
    elif P_1==maxp:
        test_data.at[i, "Predict"] = class_labels[1]
    else:
        test_data.at[i, "Predict"] = class_labels[2]

In [15]:
train_accuracy = len(train_data[train_data["Class"]==train_data["Predict"]]) / len(train_data)
test_accuracy = len(test_data[test_data["Class"]==test_data["Predict"]]) / len(test_data)

print("Train Accuracy:    ", train_accuracy * 100)
print("Test Accuracy:    ", test_accuracy * 100)

Train Accuracy:     96.19047619047619
Test Accuracy:     95.55555555555556


As we can see train accuracy is 96.19% and test accuracy is 95.55%

In [17]:
new_class_0 = train_data.loc[train_data["Class"] == class_labels[0]]
new_class_1 = train_data.loc[train_data["Class"] == class_labels[1]]
new_class_2 = train_data.loc[train_data["Class"] == class_labels[2]]


class0_p0 = len(new_class_0[new_class_0["Predict"]==class_labels[0]])
class0_p1 = len(new_class_0[new_class_0["Predict"]==class_labels[1]])
class0_p2 = len(new_class_0[new_class_0["Predict"]==class_labels[2]])

class1_p0 = len(new_class_1[new_class_1["Predict"]==class_labels[0]])
class1_p1 = len(new_class_1[new_class_1["Predict"]==class_labels[1]])
class1_p2 = len(new_class_1[new_class_1["Predict"]==class_labels[2]])

class2_p0 = len(new_class_2[new_class_2["Predict"]==class_labels[0]])
class2_p1 = len(new_class_2[new_class_2["Predict"]==class_labels[1]])
class2_p2 = len(new_class_2[new_class_2["Predict"]==class_labels[2]])


In [21]:
print()
print("Confusion Matrix for Train data:")
print()
print("Predicted", "      ",class_labels[0], "      ",class_labels[1], "      ",class_labels[2])
print("Actual") 
print() 
print(class_labels[0], "        ", class0_p0, "        ", class0_p1, "        ", class0_p2)
print(class_labels[1], "     ", class1_p0, "        ", class1_p1, "        ", class1_p2)
print(class_labels[2], "      ", class2_p0, "        ", class2_p1, "        ", class2_p2)


Confusion Matrix for Train data:

Predicted        Iris-setosa        Iris-versicolor        Iris-virginica
Actual

Iris-setosa          40          0          0
Iris-versicolor       0          30          2
Iris-virginica        0          2          31


In [22]:
new_class_0 = test_data.loc[test_data["Class"] == class_labels[0]]
new_class_1 = test_data.loc[test_data["Class"] == class_labels[1]]
new_class_2 = test_data.loc[test_data["Class"] == class_labels[2]]

class0_p0 = len(new_class_0[new_class_0["Predict"]==class_labels[0]])
class0_p1 = len(new_class_0[new_class_0["Predict"]==class_labels[1]])
class0_p2 = len(new_class_0[new_class_0["Predict"]==class_labels[2]])

class1_p0 = len(new_class_1[new_class_1["Predict"]==class_labels[0]])
class1_p1 = len(new_class_1[new_class_1["Predict"]==class_labels[1]])
class1_p2 = len(new_class_1[new_class_1["Predict"]==class_labels[2]])

class2_p0 = len(new_class_2[new_class_2["Predict"]==class_labels[0]])
class2_p1 = len(new_class_2[new_class_2["Predict"]==class_labels[1]])
class2_p2 = len(new_class_2[new_class_2["Predict"]==class_labels[2]])



print()
print("Confusion Matrix for Test data:")
print()
print("Predicted", "      ",class_labels[0], "      ",class_labels[1], "      ",class_labels[2])
print("Actual") 
print() 
print(class_labels[0], "        ", class0_p0, "        ", class0_p1, "        ", class0_p2)
print(class_labels[1], "     ", class1_p0, "        ", class1_p1, "        ", class1_p2)
print(class_labels[2], "      ", class2_p0, "        ", class2_p1, "        ", class2_p2)


Confusion Matrix for Test data:

Predicted        Iris-setosa        Iris-versicolor        Iris-virginica
Actual

Iris-setosa          10          0          0
Iris-versicolor       0          18          0
Iris-virginica        0          2          15


Now we classify the data using python libraries

In [23]:
Y = data_frame["Class"]
X = data_frame.drop(["Class", "Predict"], axis = 1)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=0)

In [24]:
model = GaussianNB()
model.fit(X_train, Y_train.values.ravel())

GaussianNB()

In [25]:
Y_predict_train = model.predict(X_train)
Y_predict_test = model.predict(X_test)

print(f"Train Accuracy: {model.score(X_train, Y_train) * 100}")
print(f"Test Accuracy : {model.score(X_test, Y_test) * 100}")

Train Accuracy: 94.28571428571428
Test Accuracy : 100.0


As we can see accuracy of train data is 94.28% and accurcy of test data is 100%

In [26]:
print("Train Data: ")
print(classification_report(Y_train, Y_predict_train))
print("*********************************************************")
print("Test Data: ")
print(classification_report(Y_test, Y_predict_test))

Train Data: 
                 precision    recall  f1-score   support

    Iris-setosa       1.00      1.00      1.00        34
Iris-versicolor       0.91      0.91      0.91        32
 Iris-virginica       0.92      0.92      0.92        39

       accuracy                           0.94       105
      macro avg       0.94      0.94      0.94       105
   weighted avg       0.94      0.94      0.94       105

*********************************************************
Test Data: 
                 precision    recall  f1-score   support

    Iris-setosa       1.00      1.00      1.00        16
Iris-versicolor       1.00      1.00      1.00        18
 Iris-virginica       1.00      1.00      1.00        11

       accuracy                           1.00        45
      macro avg       1.00      1.00      1.00        45
   weighted avg       1.00      1.00      1.00        45

